In [8]:
import gzip

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [12]:
from collections import defaultdict
import numpy as np
data = []
count = 0;

for l in readGz("train.json.gz"):
    data.append(l)

trainData, validData = np.array_split(data, 2)

print(len(trainData))
print(len(validData))
print(trainData[0])

100000
100000
{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}


In [28]:
businessCount = defaultdict(int)
visitedBusinesses = defaultdict(int)
users = defaultdict(int)
totalVisits = 0;

for d in trainData:
    user, business = d['userID'], d['businessID']
    businessCount[business] += 1
    totalVisits += 1
    visitedBusinesses[user+'-'+business] += 1
    users[user] += 1

print(len(businessCount))
print(len(users))
print(len(visitedBusinesses))

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalVisits/2: break

20490
18052
100000


In [34]:
businessList = []
usersList = []

'''for c in businessCount:
    businessList.append(c)
for u in users:
    usersList.append(u)''' #Not taking the pairs from training data, taking them from validation data

vbusinessCount = defaultdict(int)
vvisitedBusinesses = defaultdict(int)
vusers = defaultdict(int)
vtotalVisits = 0;
vnotvisited = defaultdict(int)

for d in validData:
    user, business = d['userID'], d['businessID']
    vbusinessCount[business] += 1
    vtotalVisits += 1
    vvisitedBusinesses[user+'-'+business] += 1
    vusers[user] += 1
    
print(len(vvisitedBusinesses))
print(len(vbusinessCount))
print(len(vusers))

for c in vbusinessCount:
    businessList.append(c)
for u in vusers:
    usersList.append(u)

import random

while(len(vnotvisited)<100000):
    pair = random.choice(usersList)+'-'+random.choice(businessList)
    while(pair in vvisitedBusinesses):
        pair = random.choice(usersList)+'-'+random.choice(businessList)
    vnotvisited[pair] += 1

print(len(vnotvisited))

100000
20521
18079
100000


In [38]:
wrongPred = 0;
correctPred = 0;

def baselinePredict(userBusinessPair, visited):
    global wrongPred
    global correctPred
    for p in userBusinessPair:
        u,b = p.strip().split('-')
        if((b in return1 and visited) or (b not in return1 and not visited)):
            correctPred += 1
        else:
            wrongPred += 1

baselinePredict(vvisitedBusinesses, True)
baselinePredict(vnotvisited, False)

print(wrongPred)
print(correctPred)

75631
124369
